In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.svm import SVC

target = 'moved_after_2019'
idx = 'user_id'
root = '/Users/senaemirler/garanti-bbva-data-camp/'


In [2]:
#verisetlerin,i yüklüyoruz
df_train = pd.read_csv(os.path.join(root, 'train_users.csv'))
df_test  = pd.read_csv(os.path.join(root, 'test_users.csv'))
df_subm  = pd.read_csv(os.path.join(root, 'submission.csv'))
df_lang  = pd.read_csv(os.path.join(root, 'languages.csv'))
df_edu   = pd.read_csv(os.path.join(root, 'education.csv'))
df_skill = pd.read_csv(os.path.join(root, 'skills.csv'))
df_exp   = pd.read_csv(os.path.join(root, 'work_experiences.csv'))

In [3]:
df_edu = df_edu.applymap(lambda x: x.lower() if type(x) == str else x)
df_train = df_train.applymap(lambda x: x.lower() if type(x) == str else x)
df_test = df_test.applymap(lambda x: x.lower() if type(x) == str else x)
df_skill= df_skill.applymap(lambda x: x.lower() if type(x) == str else x)
df_lang = df_lang.applymap(lambda x: x.lower() if type(x) == str else x)
df_exp = df_exp.applymap(lambda x: x.lower() if type(x) == str else x)

In [ ]:
#df_edu = df_edu.apply(lambda x: x.str.lower() if x.dtype == "object" else x)


In [4]:
df_edu.fillna({'fields_of_study':'other'}, inplace=True)

In [5]:
df_edu.loc[~df_edu['school_name'].str.contains('lisans|üniversite|university|univercity|uni|üni', case=False), 'school_name'] = 'other'


In [6]:
df_edu.loc[df_edu['school_name'].str.contains("kocaeli|gebze", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "kocaeli üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("fırat|firat", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "fırat üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("i̇stanbul teknik|i̇stanbul tecnic|itü|ıtu|ıtü|itu|istanbul technical university", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "istanbul teknik üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("marmara", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "marmara üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("anadolu university|anadolu üniversitesi", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "anadolu üniversitesi"

df_edu.loc[df_edu['school_name'].str.contains("sakarya", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "sakarya üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("yıldız teknik|ytü|ytu|yildiz", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "yıldız teknik üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("i̇stanbul teknik|i̇stanbul tecnic|itü|ıtu|ıtü|itu", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "istanbul teknik üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("hacettepe", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "hacettepe üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("ege", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "ege üniversitesi"

df_edu.loc[df_edu['school_name'].str.contains("istanbul üniversitesi|istanbul university", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "istanbul üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("osmangazi", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "osmangazi üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("gazi|gazı", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "gazi üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("karadeniz", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "karadeniz teknik üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("eylül|eylul", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "dokuz eylül üniversitesi"

df_edu.loc[df_edu['school_name'].str.contains("bahcesehir|bahçeşehir", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "bahçeşehir üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("selçuk", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "selçuk üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("beykent", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "beykent üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("ankara üniversitesi|ankara universitesi|ankara univercity|ankara university", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "ankara üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("trakya", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "trakya üniversitesi"

df_edu.loc[df_edu['school_name'].str.contains("süleyman|suleyman", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "süleyman demirel üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("boğaziçi university|boğaziçi üniversitesi|boun|boün|bogaz", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "boğaziçi üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("bilkent üniversitesi|bilkent university|bilkent", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "bilkent üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("ankara üniversitesi|ankara universitesi|ankara univercity|ankara university", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "ankara üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("erciyes", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "erciyes üniversitesi"

df_edu.loc[df_edu['school_name'].str.contains("çukurova|cukurova", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "çukurova üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("tobb", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "tobb üniversitesi"


df_edu.loc[df_edu['school_name'].str.contains("metu|odtü|middle east|orta doğu| orta dogu", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "orta doğu teknik üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("yeditepe", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "yeditepe üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("sabanci|sabancı", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "sabancı üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("mugla|muğla", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "muğla üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("akdeniz", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "akdeniz üniversitesi"


df_edu.loc[df_edu['school_name'].str.contains("koç|koc", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "koç üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("ahmet yesevi", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "ahmet yesevi üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("istanbul bilgi|i̇stanbul bilgi", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "istanbul bilgi üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("atatürk", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "atatürk üniversitesi"

df_edu.loc[df_edu['school_name'].str.contains("eskişehir|eskisehir", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "eskişehir üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("başkent|baskent", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "başkent üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("galatasaray", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "galatasaray üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("uludağ|uludag", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "atatürk üniversitesi"

df_edu.loc[df_edu['school_name'].str.contains("atılım|atilim", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "atılım üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("celal", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "celal bayar üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("galatasaray", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "galatasaray üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("uludağ|uludag", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "atatürk üniversitesi"




In [7]:
top_30_school_names = df_edu['school_name'].value_counts().nlargest(60)
print(top_30_school_names)

other                                28120
istanbul teknik üniversitesi          8079
anadolu üniversitesi                  8052
koç üniversitesi                      7416
yıldız teknik üniversitesi            6614
gazi üniversitesi                     5936
sakarya üniversitesi                  5350
marmara üniversitesi                  5055
i̇stanbul üniversitesi                4891
hacettepe üniversitesi                4139
ege üniversitesi                      3964
bahçeşehir üniversitesi               3574
dokuz eylül üniversitesi              3152
boğaziçi üniversitesi                 3048
bilkent üniversitesi                  2848
istanbul üniversitesi                 2500
ankara üniversitesi                   2380
beykent üniversitesi                  2210
trakya üniversitesi                   2124
süleyman demirel üniversitesi         2036
karadeniz teknik üniversitesi         2023
selçuk üniversitesi                   1932
fırat üniversitesi                    1740
orta doğu t

In [8]:
numerical_cols = df_edu.select_dtypes(include='number').columns
print(numerical_cols)


Index(['user_id', 'start_year_month', 'end_year_month'], dtype='object')


In [9]:
#df_edu['start_year_month'].interpolate(method='linear', inplace=True)


In [10]:
#df_edu['end_year_month'].interpolate(method='linear', inplace=True)


In [11]:
numerical_cols = df_exp.select_dtypes(include='number').columns
print(numerical_cols)


Index(['user_id', 'company_id', 'start_year_month'], dtype='object')


In [12]:
#df_exp['start_year_month'].interpolate(method='linear', inplace=True)


In [13]:
#df_exp['company_id'].interpolate(method='linear', inplace=True)


In [14]:
df_train = df_train.set_index(idx)
df_test = df_test.set_index(idx)
df_subm = df_subm.set_index(idx)

df_exp = df_exp[df_exp['start_year_month'] < 201901] # see https://www.kaggle.com/competitions/garanti-bbva-data-camp/discussion/383774

In [15]:

# Define the list of terms to check for in the school_name column
#list_bachelor = ['üniversite', 'university', 'uni', 'üni', 'fakülte', 'faculty', 'enstitü', 'enstitude']


# Create a boolean mask to filter rows with the terms in my_list in the school_name column
#mask = df_edu['school_name'].str.contains('|'.join(list_bachelor))

# Set the value of the degree column to 'bachelor' for the filtered rows with a blank degree
#df_edu.loc[mask & df_edu['degree'].isnull(), 'degree'] = 'graduate'

In [16]:
#df_edu.fillna({'start_year_month':'0'}, inplace=True)
#df_edu.fillna({'fields_of_study':'0'}, inplace=True)
#df_test['industry'].fillna('unknown', inplace=True)
#df_edu.fillna({'end_year_month':'0'}, inplace=True)
#df_edu.fillna({'skill':'0'}, inplace=True)
#df_edu.fillna({'language':'0'}, inplace=True)
#df_edu.fillna({'proficiency':'0'}, inplace=True )


In [17]:
df_edu.fields_of_study

0                                            other
1                                            other
2                                            other
3                             yazılım mühendisliği
4                             yazılım mühendisliği
                            ...                   
142570                            computer science
142571                                   chemistry
142572    electronic and communication engineering
142573      electrical and electronics engineering
142574                                       other
Name: fields_of_study, Length: 142575, dtype: object

In [18]:
df_lang.fillna(0, inplace=True)


In [19]:
df_edu.fillna({'degree':'other'}, inplace=True)

In [20]:
df_test

,industry,location
user_id,,
17449,research,turkey
33967,computer software,"istanbul, istanbul, turkey"
2110,automotive,turkey
55082,internet,turkey
37165,electrical/electronic manufacturing,turkey
...,...,...
32847,computer software,turkey
20054,hospital & health care,turkey
7029,wireless,"istanbul, turkey"


In [21]:
df_test.fillna('0', inplace=True)


In [22]:
df_skill.fillna('0', inplace=True)


In [23]:
#df_lang.fillna(method='bfill', inplace=True)

In [24]:
#df_exp.fillna(method='bfill', inplace=True)

In [25]:
#df_edu['start_year_month'].fillna(method='bfill', inplace=True)

In [26]:
#df_edu.fillna({'end_year_month':'-1'}, inplace=True)
#df_edu['start_year_month'].fillna(method='bfill', inplace=True)

In [27]:
#df_edu.loc[df_edu['degree'].str.contains("Doctor|Doktor|Ph|doktora|doctor|phd|ph.d|doktora|doktora (dr.)|philosophy|ph.d.|doctor|phd|ph.d|phd of|doctor of philosophy|ph.d.", na=False), 'degree'] = "phd"
#df_edu.loc[df_edu['degree'].str.contains("yüksek|master|M.Sc|MS|MBA|M.S|masterc|master.|yüksek|master|yuksek|mba|m.s.|M.Sc|MS|ma|m.a.|MBA|M.S|master of|m.sc.|ms|m.s|m.sc|bcomaster|Yüksek|Master|MSc|MS|M.Sc.|MBA|Msc|M.Sc|M.S.|M.S", na=False), 'degree'] = "master"
#df_edu.loc[df_edu['degree'].str.contains("BS|Bs|Bachelor|BSc|BE|B.E.|B.Sc.|B.S.|B.S|Engineer|BA|BBA|BEng|B.B.A.|B.A.Sc.|Undergraduate|Licentiate|Licence|License|Lisans|lisans|Bacheleer|bachelor|Bsc|lisans|licence|bachelor|BS|B.S|B.Sc|engineer|mühendis|mezun|graduate|licentiate|ba|b. s.|b. sc.|degre|forma|associate|li̇sans|tech|web|scien|certif|bachelorlor|myo|hon|bi̇li̇şi̇m|bilşim|bilişim|bache|lisa|faculty|licence degree|mezunu.|mühendisligi|sınıf|1.|2.|3.|4.|hazırlık|hazirlik|fakültesi|fakulte|fakülte|faculty|lisans|university|üniversity|üniversite|licence|bs|b.s|b.sc|engineer|mühendis|muhendis|mezun|graduate|licentiate|bacheloru|bachelornce|fakulty|ĺisans|bachelorter|meslek yuksek|bilgisayar|bachelorsi|licance|m.y.o|license|bachelorns|bachelorsinif|erasmus|yazılım|comp.|eng|computer", na=False), 'degree'] = "graduate"
#df_edu.loc[~df_edu["degree"].isin(["phd","master","graduate"]), "degree"] = "0"


In [28]:
edu_conv = {
    "(?i).*(doktora|doctor|phd|ph.d|doktora|doktora (dr.)|philosophy|ph.d.|doctor|phd|ph.d|phd of|doctor of philosophy|ph.d.).*": "phd",
    "(?i).*(yüksek|master|M.Sc|MS|MBA|M.S|masterc|master.|yüksek|master|yuksek|mba|m.s.|M.Sc|MS|ma|m.a.|MBA|M.S|master of|m.sc.|ms|m.s|m.sc|bcomaster').*": "master",
    "(?i).*(lisans|licence|bachelor|BS|B.S|B.Sc|engineer|mühendis|mezun|graduate|licentiate|ba|b. s.|b. sc.|degre|forma|associate|li̇sans|tech|web|scien|certif|bachelorlor|myo|hon|bi̇li̇şi̇m|bilşim|bilişim|bache|lisa|faculty|licence degree|mezunu.|mühendisligi|sınıf|1.|2.|3.|4.|hazırlık|hazirlik|fakültesi|fakulte|fakülte|faculty|lisans|university|üniversity|üniversite|licence|bs|b.s|b.sc|engineer|mühendis|muhendis|mezun|graduate|licentiate|bacheloru|bachelornce|fakulty|ĺisans|bachelorter|meslek yuksek|bilgisayar|bachelorsi|licance|m.y.o|license|bachelorns|bachelorsinif|erasmus|yazılım|comp.|eng|computer).*": "graduate",
}



df_edu.loc[:, 'degree'] = df_edu.loc[:, 'degree'].replace(edu_conv, regex=True)
df_edu = df_edu[df_edu['degree'].isin(edu_conv.values())]

In [29]:
#df_edu.loc[~df_edu["degree"].isin(["phd","master","graduate"]), "degree"] = "other"

In [30]:
df_edu.fillna({'fields_of_study':'other'}, inplace=True)

In [31]:
import re

df_edu['fields_of_study'] = df_edu['fields_of_study'].apply(lambda x: 'other' if re.search('numerical|öğrenci|hazırlık|öğrenci|i̇ngilizce hazırlık|öğrenci|eşit ağırlık|sayisal|mf|sayısal|mba|bachelor|phd|master|lise|high school', str(x).lower()) else x)

In [32]:
fields_of_study_conv = {
    "(?i).*(otomotiv).*": "otomotiv sektörü",
    "(?i).*(informatik|siber|bilgi teknolojisi|game|bilgi teknolojileri|information security|i̇nternet|android|cyber|internet|yazılım|bilişim|ctis|bi̇li̇şi̇m|informatics|game design|data|computational|yazilim|software|information systems|bilgisayar|computer|sofware|veri|web|bilişim teknolojileri|yönetim bilişim|bilişim sistemleri|grafik tasarım|oyun|information technology|information technologies|management information|yönetim bilgi sistemleri).*": "yazılım sektörü",
    "(?i).*(elektrik|electric|elektronik|electrical|electronic|elektrotech|elektric|mekatronik|mechatronic).*": "elektrik sektörü",
    "(?i).*(telekom|telecom).*": "telekom sektörü",
    "(?i).*(sinyal|signal).*": "sinyal işleme",
    "(?i).*(mechanical|mekanik|control|kontrol ve otomasyon|malzeme).*": "mekanik sektörü",
    "(?i).*(civil|industrial engineer|endüstri mühe).*": "endüstri mühendisliği",
    "(?i).*(mis|yönetim ve|işletme|i̇şletme|business|isletme|political|bilgi yönetimi|technology management|engineering management|kamu yönetimi|mühendislik yönetimi|management|kalite yönetimi|yönetim).*": "işletme ve yönetim",
    "(?i).*(economic|economy|ekonomi|maliye|iktisat|i̇ktisat|economet|finance|finans|ekonometri).*": "ekonomi ve finans ",
    "(?i).*(math|matematik|i̇statistik|istatistik).*": "matematik",
    "(?i).*(makine|makina|machine).*": "makine mühendisliği",
    "(?i).*(chemical|kimya|chemistry).*": "kimya sektörü",
    "(?i).*(i̇nşaat|inşaat|construction|constuc|i̇nşaat|mimar|architect).*": "inşaat sektörü",
    "(?i).*(biyomedikal|doctor|pharmacy|eczacılık|doktor|psychology|tıp|medicine|sağlık|health|veter).*": "sağlık sektörü",
    "(?i).*(sanat|art|graphic design).*": "sanat",
    "(?i).*(education|teacher|öğrenim|öğretim|eğitim|history|philosophy|tarih|felsefe|geography).*": "eğitim öğretim",
    "(?i).*(turk|türk|turkish|türkçe|edebiyat|dilbilim).*": "türkçe",
    "(?i).*(advert|reklam|communication design and management|reklam).*": "reklam",
    "(?i).*(yabancı dil|english|ingilizce|ingiliz|ing|eng|mütercim|foreign languages|i̇ngilizce).*": "ingilizce",
    "(?i).*(physics|fizik|fen|astronomy|biyoloji|biology|biyoteknoloji|physical).*": "fen bilimleri",
    "(?i).*(law|hukuk|adalet|justice).*": "hukuk",
    "(?i).*(trade|pazarlama|ticaret|ticaret ).*": "ticaret",
    
}

df_edu.loc[:, 'fields_of_study'] = df_edu.loc[:, 'fields_of_study'].replace(fields_of_study_conv, regex=True)
df_edu = df_edu[df_edu['fields_of_study'].isin(fields_of_study_conv.values())]



In [33]:
# 'fields_of_study' sütunundaki değerlerin sayısını sayma
counts = df_edu['fields_of_study'].value_counts()

# 20'den az tekrarlanan değerleri seçme
rare_values = counts[counts < 25].index.tolist()

# 'other' olarak değiştirme
df_edu.loc[df_edu['fields_of_study'].isin(rare_values), 'fields_of_study'] = 'other'

In [34]:
df_edu.fillna(0, inplace=True)


In [35]:
def get_degree_category(degree):
    if degree.lower().startswith('graduate'):
        return 'graduate'
    elif degree.lower().startswith('phd'):
        return 'phd'
    elif degree.lower().startswith('master'):
        return 'master'
    else:
        return 'other'

df_edu['new_degree_category'] = df_edu['degree'].apply(get_degree_category)

new_edu = df_edu.groupby('user_id')['new_degree_category'].value_counts().unstack(fill_value=0).reset_index()
new_edu.columns.name = None
new_edu.columns = ['user_id', 'graduate','phd', 'master']


In [36]:
new_edu

,user_id,graduate,phd,master
0,1,1,1,0
1,2,1,0,0
2,3,1,0,0
3,4,1,0,0
4,5,2,1,0
...,...,...,...,...
53660,66269,1,1,0
53661,66270,2,2,0
53662,66271,1,1,1
53663,66272,1,0,0


In [37]:
df_edu = df_edu.drop_duplicates(['user_id', 'degree'])

df_edu = df_edu.drop('start_year_month', axis=1)
df_edu = df_edu.drop('end_year_month', axis=1)
df_edu = df_edu.drop('new_degree_category', axis=1)


In [38]:
df_edu

,user_id,school_name,degree,fields_of_study
3,1,fırat üniversitesi,master,yazılım sektörü
4,1,fırat üniversitesi,graduate,yazılım sektörü
5,2,fırat üniversitesi,graduate,yazılım sektörü
7,3,fırat üniversitesi,graduate,yazılım sektörü
8,4,fırat üniversitesi,graduate,yazılım sektörü
...,...,...,...,...
142567,66271,gazi üniversitesi,graduate,yazılım sektörü
142568,66271,marmara üniversitesi,phd,yazılım sektörü
142569,66272,yıldız teknik üniversitesi,graduate,yazılım sektörü
142572,66273,istanbul teknik üniversitesi,graduate,elektrik sektörü


In [39]:
df_lang = df_lang[df_lang['language'].notnull() & df_lang['proficiency'].notnull()]


In [40]:
lang_rank = {
    'elementary': 1,
    'limited_working': 2,
    'professional_working': 3,
    'full_professional': 4,
    'native_or_bilingual': 5
}

df_lang.loc[:, 'proficiency'] = df_lang.loc[:, 'proficiency'].map(lang_rank)

In [41]:
lang_conv = {
    '(?i).*(ingilizce|english|englishch|englisch|Eng|eng|İng|ing|Ing|English|english|İngilizce|ingilizce|Ingilizce|İNGİLİZCE|INGILIZCE).*': 'english',
    '(?i).*(almanca|german|deutsch|Alm|alm|ger|Ger|Deu|deu|ALMANCA|GER|DEU|ALM).*': 'german',
    '(?i).*(fransızca|french|français|Fr|fr|FR).*': 'french',
    '(?i).*(ispanyolca|spanish|español|Spa|spa|İsp|isp|SPA|ISP|İSP).*': 'spanish',
    '(?i).*(arapça|arabic|ara|Ara|ARA).*': 'arabic',
    '(?i).*(rusça|russian|Rus|rus|RUS).*': 'russian',
    '(?i).*(Chinese|CHINESE|CHI|Chi|chi|Çince).*': 'chinese',
    '(?i).*(Japanese|Japonca|japa|japo|JAPA|JAPO).*': 'japanese',
    '(?i).*(İtalyanca|Italian|itali|ital|ITA|İTAL).*': 'italian',
    '(?i).*(Azerice|Azerbaijani|Azeri|Azərba).*': 'azerbaijani',
    '(?i).*(türkçe|turkish|türkisch|Tür|Tur|tür|tur|TURKISH|TÜRKÇE|Tükçe).*': 'turkish',
}

df_lang['language'] = df_lang['language'].replace(lang_conv, regex=True)
df_lang = df_lang[df_lang['language'].isin(lang_conv.values())]

In [42]:
df_lang = df_lang.drop_duplicates(['user_id', 'language'])
df_lang = pd.pivot(df_lang, index='user_id', columns='language', values='proficiency')
df_lang = df_lang.fillna(0).astype(int)
df_lang.head()

language,arabic,azerbaijani,chinese,english,french,german,italian,japanese,russian,spanish,turkish
user_id,,,,,,,,,,,
8,0,0,0,0,1,0,0,0,0,0,5
10,0,0,0,0,0,0,0,0,0,0,0
11,0,0,0,3,0,0,0,0,0,0,5
12,0,0,0,3,0,0,0,0,0,0,5
13,0,0,0,0,0,2,0,0,0,0,0


In [43]:
mask = df_skill['skill'].str.contains('asp')

df_skill.loc[mask, 'skill'] = 'asp' 

mask = df_skill['skill'].str.contains('html')

df_skill.loc[mask, 'skill'] = 'html' 
mask = df_skill['skill'].str.contains('oop')

df_skill.loc[mask, 'skill'] = 'oop' 
mask = df_skill['skill'].str.contains('py')

df_skill.loc[mask, 'skill'] = 'python' 

mask = df_skill['skill'].str.contains('agile')

df_skill.loc[mask, 'skill'] = 'agile' 
mask = df_skill['skill'].str.contains('sql')

df_skill.loc[mask, 'skill'] = 'sql' 
mask = df_skill['skill'].str.contains('php')

df_skill.loc[mask, 'skill'] = 'php' 

mask = df_skill['skill'].str.contains('node')

df_skill.loc[mask, 'skill'] = 'node' 

mask = df_skill['skill'].str.contains('android')

df_skill.loc[mask, 'skill'] = 'android' 
mask = df_skill['skill'].str.contains('microsoft')

df_skill.loc[mask, 'skill'] = 'microsoft' 
mask = df_skill['skill'].str.contains('react')

df_skill.loc[mask, 'skill'] = 'react' 
mask = df_skill['skill'].str.contains('proje yönetimi')

df_skill.loc[mask, 'skill'] = 'project management' 
mask = df_skill['skill'].str.contains('web geliştirme')

df_skill.loc[mask, 'skill'] = 'web development' 
mask = df_skill['skill'].str.contains('spring')

df_skill.loc[mask, 'skill'] = 'spring' 



In [44]:
used_skills = df_skill['skill'].value_counts().iloc[:65].index # we only took most common 60 skills, you can increase it

df_skill = df_skill[df_skill['skill'].isin(used_skills)]
df_skill['experience'] = True

In [45]:
df_skill = df_skill.drop_duplicates(['user_id', 'skill'])
df_skill = pd.pivot(df_skill, index='user_id', columns='skill', values='experience')
df_skill = df_skill.fillna(0).astype(int)
df_skill.head()

skill,.net,.net core,agile,ajax,android,angular,araştırma,asp,autocad,bootstrap,...,software project management,spring,sql,tfs,typescript,visual studio,web development,web services,xml,yazılım geliştirme
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
df_exp['location'].fillna('unknown', inplace=True)

In [47]:
df_exp.loc[df_exp['location'].str.contains("ankara|odtü|metu|hacettepe|bilkent", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "ankara"
df_exp.loc[df_exp['location'].str.contains("izmir|i̇zmir,|i̇zmir|i̇zmi̇r", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "izmir"
df_exp.loc[df_exp['location'].str.contains("gayrettepe|mecidiyekoy|ayazağa|yıldız|küçükyalı|maltepe|itu|itü|taksim|levent|çerkezköy|kozyatağı|marmara|kadıköy|istanbul|pendik|tuzla|çekmeköy|i̇stanbul|istanbul,|istanb|üsküdar|beyoğlu|yıldız teknik|ytü|kartal|maslak|ümraniye|beşiktaş|besiktas|ataşehir", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "istanbul"
df_exp.loc[df_exp['location'].str.contains("adana", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "adana"
df_exp.loc[df_exp['location'].str.contains("kocaeli|izmit|gebze|gölcük", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "kocaeli"
df_exp.loc[df_exp['location'].str.contains("bursa|gemlik", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "bursa"
df_exp.loc[df_exp['location'].str.contains("eskisehir|eskişehir", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "eskisehir"
df_exp.loc[df_exp['location'].str.contains("sakarya", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "sakarya"
df_exp.loc[df_exp['location'].str.contains("manisa", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "manisa"
df_exp.loc[df_exp['location'].str.contains("konya", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "konya"
df_exp.loc[df_exp['location'].str.contains("kayseri", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "kayseri"
df_exp.loc[df_exp['location'].str.contains("antalya", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "antalya"
df_exp.loc[df_exp['location'].str.contains("isparta|ısparta", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "ısparta"
df_exp.loc[df_exp['location'].str.contains("elazig|elazığ|elazı", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "elazığ"
df_exp.loc[df_exp['location'].str.contains("mugla|muğla|bodrum|marmaris", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "bodrum"
df_exp.loc[df_exp['location'].str.contains("mersin", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "mersin"
df_exp.loc[df_exp['location'].str.contains("francisco", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "francisco"
df_exp.loc[df_exp['location'].str.contains("trabzon", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "trabzon"
df_exp.loc[df_exp['location'].str.contains("tekirda", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "tekirdağ"
df_exp.loc[df_exp['location'].str.contains("denizli", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "denizli"
df_exp.loc[df_exp['location'].str.contains("samsun", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "samsun"
df_exp.loc[df_exp['location'].str.contains("antep", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "antep"
df_exp.loc[df_exp['location'].str.contains("texas|kansas|united states|america|amerika|abd|new york|florida", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "abd"
df_exp.loc[df_exp['location'].str.contains("italy|i̇talya|italya", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "italy"
df_exp.loc[df_exp['location'].str.contains("yalova", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "yalova"
df_exp.loc[df_exp['location'].str.contains("germany|almanya|alman", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "germany"
df_exp.loc[df_exp['location'].str.contains("balikesir|balıkesir", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "balıkesir"
df_exp.loc[df_exp['location'].str.contains("pakistan", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "pakistan"
df_exp.loc[df_exp['location'].str.contains("united kingdom|maidenhead|uk|england|ingiltere", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "edirne"
df_exp.loc[df_exp['location'].str.contains("nevşehir|nevsehir", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "nevşehir"
df_exp.loc[df_exp['location'].str.contains("poland", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "poland"
df_exp.loc[df_exp['location'].str.contains("staj|intern", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "staj"
df_exp.loc[df_exp['location'].str.contains("remote|uzaktan", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "remote"
df_exp.loc[df_exp['location'].str.contains("edirne", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "edirne"
df_exp.loc[df_exp['location'].str.contains("düzce|duzce", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "düzce"
df_exp.loc[df_exp['location'].str.contains("sales", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "sales"
df_exp.loc[df_exp['location'].str.contains("sivas", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "sivas"
df_exp.loc[df_exp['location'].str.contains("hatay", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "hatay"
df_exp.loc[df_exp['location'].str.contains("spain", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "spain"
df_exp.loc[df_exp['location'].str.contains("kirklareli|kırklareli|lüleburgaz", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "kırklareli"
df_exp.loc[df_exp['location'].str.contains("nether", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "netherlands"
df_exp.loc[df_exp['location'].str.contains("malatya", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "malatya"
df_exp.loc[df_exp['location'].str.contains("duzce|düzce", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "duzce"
df_exp.loc[df_exp['location'].str.contains("software", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "software"
df_exp.loc[df_exp['location'].str.contains("maraş|maras| maraş", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "maraş"
df_exp.loc[df_exp['location'].str.contains("aydin|aydın", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "aydın"
df_exp.loc[df_exp['location'].str.contains("zonguldak", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "zonguldak"
df_exp.loc[df_exp['location'].str.contains("erzurum", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "erzurum"
df_exp.loc[df_exp['location'].str.contains("bolu", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "bolu"
df_exp.loc[df_exp['location'].str.contains("france|paris", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "france"
df_exp.loc[df_exp['location'].str.contains("azerbaijan", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "azerbaijan"
df_exp.loc[df_exp['location'].str.contains("canakkale", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "canakkale"
df_exp.loc[df_exp['location'].str.contains("afyon", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "afyon"
df_exp.loc[df_exp['location'].str.contains("sweden", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "sweden"
df_exp.loc[df_exp['location'].str.contains("erzurum", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "erzurum"
df_exp.loc[df_exp['location'].str.contains("switzerland|isviçre", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "switzerland"
df_exp.loc[df_exp['location'].str.contains("norway|, norway", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "norway"
df_exp.loc[df_exp['location'].str.contains("van", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "van"
df_exp.loc[df_exp['location'].str.contains("russia|rusya", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "russia"
df_exp.loc[df_exp['location'].str.contains("kazak", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "kazak"
df_exp.loc[df_exp['location'].str.contains("egypt", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "egypt"
df_exp.loc[df_exp['location'].str.contains("denmark", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "denmark"
df_exp.loc[df_exp['location'].str.contains("hungary", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "hungary"
df_exp.loc[df_exp['location'].str.contains("syria", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "syria"
df_exp.loc[df_exp['location'].str.contains("portugal", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "portugal"
df_exp.loc[df_exp['location'].str.contains("cyprus|kıbrıs|kibris", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "cyprus"
df_exp.loc[df_exp['location'].str.contains("macedonia", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "macedonia"
df_exp.loc[df_exp['location'].str.contains("suudi|arabistan|saudi", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "arabistan"
df_exp.loc[df_exp['location'].str.contains("australia", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "australia"
df_exp.loc[df_exp['location'].str.contains("canada", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "canada"
df_exp.loc[df_exp['location'].str.contains("lithuania", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "lithuania"
df_exp.loc[df_exp['location'].str.contains("aksaray", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "aksaray"
df_exp.loc[df_exp['location'].str.contains("bilecik", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "bilecik"
df_exp.loc[df_exp['location'].str.contains("karabük", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "karabük"




In [48]:
keep_locations = ['ankara', 'izmir', 'istanbul', 'adana', 'kocaeli', 'bursa', 'eskisehir', 'sakarya', 'manisa', 'konya', 'kayseri', 'antalya', 'ısparta', 'elazığ', 'bodrum', 'mersin', 'francisco', 'trabzon', 'tekirdağ', 'denizli', 'samsun', 'antep', 'abd', 'italy', 'yalova', 'germany', 'balıkesir', 'pakistan', 'ingiltere', 'nevşehir', 'poland', 'staj', 'remote', 'edirne', 'düzce', 'sales', 'sivas', 'hatay', 'spain', 'kırklareli', 'netherlands', 'malatya', 'software', 'maraş', 'aydın', 'zonguldak', 'erzurum',  'bolu', 'france', 'azerbaijan', 'canakkale', 'afyon', 'sweden', 'erzurum', 'switzerland', 'norway', 'van', 'russia', 'kazak', 'egypt', 'denmark', 'hungary', 'syria', 'portugal', 'cyprus', 'macedonia', 'arabistan', 'australia', 'canada', 'lithuania', 'aksaray', 'bilecik', 'karabük']

df_exp['location'] = df_exp['location'].apply(lambda x: x if x.lower() in keep_locations else '0')


In [49]:
df_exp = df_exp.sort_values(by=['user_id', 'start_year_month']) # we want nth function to return the last companies by date
df_new = pd.DataFrame()

In [50]:
df_new['company(1th)'] = df_exp.groupby(idx)['company_id'].nth(-1).astype(str)
df_new['company(2th)'] = df_exp.groupby(idx)['company_id'].nth(-2).astype(str)
df_new['company(3th)'] = df_exp.groupby(idx)['company_id'].nth(-3).astype(str)

df_new['company_location(1th)'] = df_exp.groupby(idx)['location'].nth(-1).astype(str)
df_new['company_location(2th)'] = df_exp.groupby(idx)['location'].nth(-2).astype(str)
df_new['company_location(3th)'] = df_exp.groupby(idx)['location'].nth(-3).astype(str)

df_new['min_exp_time'] = df_exp.groupby(idx)['start_year_month'].min()
df_new['max_exp_time'] = df_exp.groupby(idx)['start_year_month'].max()

df_new['company_count_2018'] = df_exp[df_exp['start_year_month'].gt(201712)].groupby(idx).size()
df_new['company_count_2017'] = df_exp[df_exp['start_year_month'].gt(201612)].groupby(idx).size()
df_new['company_count_2016'] = df_exp[df_exp['start_year_month'].gt(201512)].groupby(idx).size()
df_new['company_count_2015'] = df_exp[df_exp['start_year_month'].gt(201412)].groupby(idx).size()


In [51]:
df_exp = df_new
df_exp.head()

,company(1th),company(2th),company(3th),company_location(1th),company_location(2th),company_location(3th),min_exp_time,max_exp_time,company_count_2018,company_count_2017,company_count_2016,company_count_2015
user_id,,,,,,,,,,,,
0,0,0,NaN,0,0,NaN,200509,200509,NaN,NaN,NaN,NaN
2,10,7,9,mersin,elazığ,elazığ,201612,201806,1.0,2.0,3.0,3.0
5,15,NaN,NaN,istanbul,NaN,NaN,201706,201706,NaN,1.0,1.0,1.0
7,20,21,21,elazığ,macedonia,istanbul,201607,201812,1.0,3.0,4.0,4.0
10,26,26,27,istanbul,istanbul,elazığ,201308,201805,2.0,2.0,2.0,2.0


In [52]:
df_train[df_edu.columns]   = df_edu[df_edu.columns]
df_train[df_lang.columns]  = df_lang[df_lang.columns]
df_train[df_skill.columns] = df_skill[df_skill.columns]
df_train[df_exp.columns]   = df_exp[df_exp.columns]

df_test[df_edu.columns]   = df_edu[df_edu.columns]
df_test[df_lang.columns]  = df_lang[df_lang.columns]
df_test[df_skill.columns] = df_skill[df_skill.columns]
df_test[df_exp.columns]   = df_exp[df_exp.columns]

In [53]:
cat_cols = [col for col in df_test.columns if df_test[col].dtype == 'object']
num_cols = [col for col in df_test.columns if df_test[col].dtype != 'object']

In [54]:
for col in cat_cols:
    train_cats  = set(df_train[col].unique())
    test_cats   = set(df_test[col].unique())
    common_cats = set.intersection(train_cats, test_cats)
    
    df_train.loc[~df_train[col].isin(common_cats), col] = 'other'
    df_test.loc[~df_test[col].isin(common_cats), col] = 'other'

In [55]:
df_all = pd.concat([df_train, df_test], axis=0)

for col in cat_cols:
    df_all[col] = df_all[col].factorize()[0]

df_all[cat_cols] = df_all[cat_cols].astype('category')
df_all[num_cols] = df_all[num_cols].fillna(-1)

df_train = df_all.loc[df_train.index, df_train.columns]
df_test = df_all.loc[df_test.index, df_test.columns]

In [56]:
df_train

,industry,location,moved_after_2019,user_id,school_name,degree,fields_of_study,arabic,azerbaijani,chinese,...,company(3th),company_location(1th),company_location(2th),company_location(3th),min_exp_time,max_exp_time,company_count_2018,company_count_2017,company_count_2016,company_count_2015
user_id,,,,,,,,,,,,,,,,,,,,,
1301,0,0,1.0,761.0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,201505.0,201803.0,1.0,1.0,1.0,3.0
6950,1,0,0.0,3597.0,1,0,1,0.0,0.0,0.0,...,1,1,1,1,201304.0,201702.0,-1.0,1.0,3.0,4.0
4880,2,1,0.0,2538.0,2,1,2,-1.0,-1.0,-1.0,...,2,1,1,2,201401.0,201705.0,-1.0,1.0,2.0,3.0
26046,3,0,0.0,-1.0,-1,-1,-1,0.0,0.0,0.0,...,3,1,1,2,200909.0,201410.0,-1.0,-1.0,-1.0,-1.0
11005,4,2,0.0,-1.0,-1,-1,-1,0.0,0.0,0.0,...,4,1,1,2,200909.0,201704.0,-1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57247,0,0,0.0,26685.0,4,0,0,-1.0,-1.0,-1.0,...,168,1,4,1,200704.0,201811.0,1.0,1.0,1.0,2.0
37784,3,2,0.0,-1.0,-1,-1,-1,0.0,0.0,0.0,...,138,1,4,1,201005.0,201408.0,-1.0,-1.0,-1.0,-1.0
33229,0,4,0.0,-1.0,-1,-1,-1,0.0,0.0,0.0,...,282,3,2,32,201002.0,201806.0,1.0,1.0,2.0,2.0


In [57]:
print(df_train.columns.tolist())

['industry', 'location', 'moved_after_2019', 'user_id', 'school_name', 'degree', 'fields_of_study', 'arabic', 'azerbaijani', 'chinese', 'english', 'french', 'german', 'italian', 'japanese', 'russian', 'spanish', 'turkish', '.net', '.net core', 'agile', 'ajax', 'android', 'angular', 'araştırma', 'asp', 'autocad', 'bootstrap', 'c', 'c#', 'c++', 'css', 'design patterns', 'docker', 'eclipse', 'ekip çalışması', 'entity framework', 'git', 'hibernate', 'html', 'i̇ngilizce', 'java', 'javascript', 'jira', 'jquery', 'json', 'liderlik', 'linq', 'linux', 'machine learning', 'management', 'matlab', 'maven', 'microsoft', 'mongodb', 'mühendislik', 'node', 'object oriented design', 'oop', 'oracle', 'photoshop', 'php', 'programlama', 'programming', 'project management', 'python', 'react', 'scrum', 'soap', 'software design', 'software development', 'software engineering', 'software project management', 'spring', 'sql', 'tfs', 'typescript', 'visual studio', 'web development', 'web services', 'xml', 'yazı

In [58]:
ftrain = ['industry', 'location', 'moved_after_2019', 'user_id', 'school_name', 'degree', 'fields_of_study', 'arabic', 'azerbaijani', 'chinese', 'english', 'french', 'german', 'italian', 'japanese', 'russian', 'spanish', 'turkish', '.net', '.net core', 'agile', 'ajax', 'android', 'angular', 'araştırma', 'asp', 'autocad', 'bootstrap', 'c', 'c#', 'c++', 'css', 'design patterns', 'docker', 'eclipse', 'ekip çalışması', 'entity framework', 'git', 'hibernate', 'html', 'i̇ngilizce', 'java', 'javascript', 'jira', 'jquery', 'json', 'liderlik', 'linq', 'linux', 'machine learning', 'management', 'matlab', 'maven', 'microsoft', 'mongodb', 'mühendislik', 'node', 'object oriented design', 'oop', 'oracle', 'photoshop', 'php', 'programlama', 'programming', 'project management', 'python', 'react', 'scrum', 'soap', 'software design', 'software development', 'software engineering', 'software project management', 'spring', 'sql', 'tfs', 'typescript', 'visual studio', 'web development', 'web services', 'xml', 'yazılım geliştirme', 'company(1th)', 'company(2th)', 'company(3th)', 'company_location(1th)', 'company_location(2th)', 'company_location(3th)', 'min_exp_time', 'max_exp_time', 'company_count_2018', 'company_count_2017', 'company_count_2016', 'company_count_2015']
ftest = ['industry', 'location', 'moved_after_2019', 'user_id', 'school_name', 'degree', 'fields_of_study', 'arabic', 'azerbaijani', 'chinese', 'english', 'french', 'german', 'italian', 'japanese', 'russian', 'spanish', 'turkish', '.net', '.net core', 'agile', 'ajax', 'android', 'angular', 'araştırma', 'asp', 'autocad', 'bootstrap', 'c', 'c#', 'c++', 'css', 'design patterns', 'docker', 'eclipse', 'ekip çalışması', 'entity framework', 'git', 'hibernate', 'html', 'i̇ngilizce', 'java', 'javascript', 'jira', 'jquery', 'json', 'liderlik', 'linq', 'linux', 'machine learning', 'management', 'matlab', 'maven', 'microsoft', 'mongodb', 'mühendislik', 'node', 'object oriented design', 'oop', 'oracle', 'photoshop', 'php', 'programlama', 'programming', 'project management', 'python', 'react', 'scrum', 'soap', 'software design', 'software development', 'software engineering', 'software project management', 'spring', 'sql', 'tfs', 'typescript', 'visual studio', 'web development', 'web services', 'xml', 'yazılım geliştirme', 'company(1th)', 'company(2th)', 'company(3th)', 'company_location(1th)', 'company_location(2th)', 'company_location(3th)', 'min_exp_time', 'max_exp_time', 'company_count_2018', 'company_count_2017', 'company_count_2016', 'company_count_2015']

In [59]:
def Definedata():
    # define dataset
    data2=df_train[ftrain]
    X=data2.drop(columns=['moved_after_2019']).values
    y=data2['moved_after_2019'].values
    return X, y

In [60]:
def SMOTE():
    # borderline-SMOTE for imbalanced dataset
    from collections import Counter
    from sklearn.model_selection import train_test_split
    from sklearn.datasets import make_classification
    from imblearn.over_sampling import SMOTE
    from matplotlib import pyplot
    from numpy import where
    
    X, y = Definedata()

# summarize class distribution
    counter = Counter(y)
    print(counter)
# transform the dataset
    smt = SMOTE(random_state=0)
    X, y = smt.fit_sample(X, y) 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)
# summarize the new class distribution
    counter = Counter(y)
    print(counter)
# scatter plot of examples by class label
    for label, _ in counter.items():
        row_ix = where(y == label)[0]
        pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
    pyplot.legend()
    pyplot.show()
    return X_train, X_test, y_train, y_test

In [ ]:
def BSMOTE():
    from collections import Counter
    from sklearn.model_selection import train_test_split
    from imblearn.over_sampling import BorderlineSMOTE
    from matplotlib import pyplot
    from numpy import where
    
    X, y = Definedata()
    
# summarize class distribution
    counter = Counter(y)
    print(counter)
# transform the dataset
    X, y = BorderlineSMOTE().fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)
# summarize the new class distribution
    counter = Counter(y)
    print(counter)
# scatter plot of examples by class label
    for label, _ in counter.items():
        row_ix = where(y == label)[0]
        pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
    pyplot.legend()
    pyplot.show()
    return X_train, X_test, y_train, y_test    

In [ ]:
def SMOTESVM():
    from collections import Counter
    from sklearn.model_selection import train_test_split
    from imblearn.over_sampling import SVMSMOTE
    from matplotlib import pyplot
    from numpy import where

    X, y = Definedata()

# summarize class distribution
    counter = Counter(y)
    print(counter)
# transform the dataset
    X, y = SVMSMOTE().fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)
# summarize the new class distribution
    counter = Counter(y)
    print(counter)
# scatter plot of examples by class label
    for label, _ in counter.items():
        row_ix = where(y == label)[0]
        pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
    pyplot.legend()
    pyplot.show()
    return X_train, X_test, y_train, y_test

In [ ]:
def ADASYN():
    from collections import Counter
    from sklearn.model_selection import train_test_split
    from imblearn.over_sampling import ADASYN
    from matplotlib import pyplot
    from numpy import where

    X, y = Definedata()

# summarize class distribution
    counter = Counter(y)
    print(counter)
# transform the dataset
    X, y = ADASYN().fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)
# summarize the new class distribution
    counter = Counter(y)
    print(counter)
# scatter plot of examples by class label
    for label, _ in counter.items():
        row_ix = where(y == label)[0]
        pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
    pyplot.legend()
    pyplot.show()
    return X_train, X_test, y_train, y_test

In [61]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier

# Define target variable name
target = 'moved_after_2019'

data2=df_train[ftrain]

# Define X and y
X, y = data2.drop(columns=[target]), data2[target]




In [62]:
# Define CatBoostClassifier with some initial hyperparameters
clf = CatBoostClassifier(n_estimators=100, learning_rate=0.1, random_seed=42)

# Define cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Initialize list to store cross-validation scores
scores = []

# Loop through each fold and train/validate the model
for train_idx, val_idx in cv.split(X, y):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    clf.fit(X_train, y_train, cat_features=cat_cols)
    scores.append(clf.score(X_val, y_val))

# Print the mean and standard deviation of the scores
print(f'Scores mean: {np.mean(scores):.4f}')
print(f'Scores std: {np.std(scores):.4f}')


0:	learn: 0.6838737	total: 85.9ms	remaining: 8.5s
1:	learn: 0.6762673	total: 106ms	remaining: 5.18s
2:	learn: 0.6694048	total: 123ms	remaining: 3.99s
3:	learn: 0.6641576	total: 140ms	remaining: 3.35s
4:	learn: 0.6595373	total: 159ms	remaining: 3.01s
5:	learn: 0.6561539	total: 176ms	remaining: 2.76s
6:	learn: 0.6520702	total: 193ms	remaining: 2.57s
7:	learn: 0.6462760	total: 209ms	remaining: 2.4s
8:	learn: 0.6414884	total: 225ms	remaining: 2.27s
9:	learn: 0.6377243	total: 242ms	remaining: 2.18s
10:	learn: 0.6341261	total: 257ms	remaining: 2.08s
11:	learn: 0.6304940	total: 274ms	remaining: 2.01s
12:	learn: 0.6279554	total: 295ms	remaining: 1.98s
13:	learn: 0.6255989	total: 314ms	remaining: 1.93s
14:	learn: 0.6234167	total: 331ms	remaining: 1.88s
15:	learn: 0.6213830	total: 349ms	remaining: 1.83s
16:	learn: 0.6188155	total: 368ms	remaining: 1.8s
17:	learn: 0.6170704	total: 385ms	remaining: 1.75s
18:	learn: 0.6150381	total: 403ms	remaining: 1.72s
19:	learn: 0.6123747	total: 420ms	remaining

70:	learn: 0.5837327	total: 1.29s	remaining: 525ms
71:	learn: 0.5834790	total: 1.3s	remaining: 507ms
72:	learn: 0.5830022	total: 1.32s	remaining: 488ms
73:	learn: 0.5828109	total: 1.34s	remaining: 472ms
74:	learn: 0.5824402	total: 1.41s	remaining: 469ms
75:	learn: 0.5821240	total: 1.43s	remaining: 450ms
76:	learn: 0.5818993	total: 1.45s	remaining: 432ms
77:	learn: 0.5817371	total: 1.46s	remaining: 412ms
78:	learn: 0.5814569	total: 1.48s	remaining: 393ms
79:	learn: 0.5812150	total: 1.5s	remaining: 374ms
80:	learn: 0.5808079	total: 1.51s	remaining: 355ms
81:	learn: 0.5805339	total: 1.53s	remaining: 336ms
82:	learn: 0.5805169	total: 1.54s	remaining: 317ms
83:	learn: 0.5800124	total: 1.56s	remaining: 298ms
84:	learn: 0.5798291	total: 1.58s	remaining: 279ms
85:	learn: 0.5794889	total: 1.6s	remaining: 260ms
86:	learn: 0.5792789	total: 1.61s	remaining: 241ms
87:	learn: 0.5790323	total: 1.63s	remaining: 222ms
88:	learn: 0.5788583	total: 1.65s	remaining: 204ms
89:	learn: 0.5786908	total: 1.66s	

38:	learn: 0.5980774	total: 911ms	remaining: 1.43s
39:	learn: 0.5975303	total: 936ms	remaining: 1.4s
40:	learn: 0.5967764	total: 960ms	remaining: 1.38s
41:	learn: 0.5962352	total: 977ms	remaining: 1.35s
42:	learn: 0.5955753	total: 995ms	remaining: 1.32s
43:	learn: 0.5952004	total: 1.01s	remaining: 1.29s
44:	learn: 0.5947960	total: 1.03s	remaining: 1.26s
45:	learn: 0.5943405	total: 1.04s	remaining: 1.23s
46:	learn: 0.5937554	total: 1.06s	remaining: 1.2s
47:	learn: 0.5934033	total: 1.08s	remaining: 1.17s
48:	learn: 0.5928199	total: 1.1s	remaining: 1.15s
49:	learn: 0.5924560	total: 1.13s	remaining: 1.13s
50:	learn: 0.5918241	total: 1.15s	remaining: 1.1s
51:	learn: 0.5916052	total: 1.17s	remaining: 1.08s
52:	learn: 0.5911511	total: 1.19s	remaining: 1.05s
53:	learn: 0.5907099	total: 1.2s	remaining: 1.02s
54:	learn: 0.5903290	total: 1.22s	remaining: 999ms
55:	learn: 0.5900296	total: 1.24s	remaining: 973ms
56:	learn: 0.5896830	total: 1.25s	remaining: 947ms
57:	learn: 0.5892940	total: 1.27s	re

In [63]:
df_subm.loc[df_test.index, target] = clf.predict(df_test)
df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()


0    9953
1    3302
Name: moved_after_2019, dtype: int64

In [64]:
df_subm.to_csv('submission.csv')

In [ ]:
data2=df_train[ftrain]
X=data2.drop(columns=['moved_after_2019']).values
y=data2['moved_after_2019'].values
clf = RandomForestClassifier(n_estimators=200, class_weight='balanced_subsample', random_state=42)
cv  = StratifiedKFold(shuffle=True, random_state=42)
scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')

In [ ]:
print(f'Scores mean: {np.mean(scores):.4f}')
print(f'Scores std: {np.std(scores):.4f}')

In [ ]:
df_subm.to_csv('submission.csv')


In [65]:
data2

,industry,location,moved_after_2019,user_id,school_name,degree,fields_of_study,arabic,azerbaijani,chinese,...,company(3th),company_location(1th),company_location(2th),company_location(3th),min_exp_time,max_exp_time,company_count_2018,company_count_2017,company_count_2016,company_count_2015
user_id,,,,,,,,,,,,,,,,,,,,,
1301,0,0,1.0,761.0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,201505.0,201803.0,1.0,1.0,1.0,3.0
6950,1,0,0.0,3597.0,1,0,1,0.0,0.0,0.0,...,1,1,1,1,201304.0,201702.0,-1.0,1.0,3.0,4.0
4880,2,1,0.0,2538.0,2,1,2,-1.0,-1.0,-1.0,...,2,1,1,2,201401.0,201705.0,-1.0,1.0,2.0,3.0
26046,3,0,0.0,-1.0,-1,-1,-1,0.0,0.0,0.0,...,3,1,1,2,200909.0,201410.0,-1.0,-1.0,-1.0,-1.0
11005,4,2,0.0,-1.0,-1,-1,-1,0.0,0.0,0.0,...,4,1,1,2,200909.0,201704.0,-1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57247,0,0,0.0,26685.0,4,0,0,-1.0,-1.0,-1.0,...,168,1,4,1,200704.0,201811.0,1.0,1.0,1.0,2.0
37784,3,2,0.0,-1.0,-1,-1,-1,0.0,0.0,0.0,...,138,1,4,1,201005.0,201408.0,-1.0,-1.0,-1.0,-1.0
33229,0,4,0.0,-1.0,-1,-1,-1,0.0,0.0,0.0,...,282,3,2,32,201002.0,201806.0,1.0,1.0,2.0,2.0


In [ ]:
from imblearn.over_sampling import SMOTE

data2 = df_train[ftrain]
X = data2.drop(columns=['moved_after_2019']).values
y = data2['moved_after_2019'].values

over = SMOTE(random_state=42)
X_train, y_train = over.fit_resample(X, y)



In [ ]:
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [ ]:
clf=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=1000, verbose=0,
            warm_start=False)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [ ]:
score = accuracy_score(y_test,y_pred)
print("Test Accuracy Score", score)


In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred_proba = lr.predict_proba(X_test)[:, 1]
y_pred_lr = lr.predict(X_test)

In [ ]:
score = accuracy_score(y_test,y_pred_lr)
print("Test Accuracy Score", score)

In [ ]:
param_grid = {
    "max_depth": [3, 4, 5, 7, 8],
    "learning_rate": [0.1, 0.01, 0.001],
    "gamma": [0, 0.25, 1],
    "reg_lambda": [0, 1, 10],
    "scale_pos_weight": [1, 3, 5],
    "subsample": [0.8],
    "colsample_bytree": [0.5],
}

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score,roc_auc_score, f1_score, plot_confusion_matrix, plot_roc_curve, plot_precision_recall_curve, classification_report, precision_score, recall_score

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV


xgb_cl = xgb.XGBClassifier(objective="binary:logistic")
grid_cv = GridSearchCV(xgb_cl, param_grid, n_jobs=-1, cv=3, scoring="roc_auc")

_ = grid_cv.fit(X_train, y_train)

In [ ]:
print(grid_cv.best_score_)


In [ ]:
y_train_pred = grid_cv.best_estimator_.predict_proba(X_train)
y_train_pred_pos = y_train_pred[:,1]

y_val_pred = grid_cv.best_estimator_.predict_proba(X_test)
y_val_pred_pos = y_val_pred[:,1]

auc_train = roc_auc_score(y_train, y_train_pred_pos)
auc_test = roc_auc_score(y_test, y_val_pred_pos)

In [ ]:
print("XGBClassifier Model Accuracy : ", round(auc_test,2))


In [ ]:
X_test = df_test.values
y_test = np.zeros(len(X_test))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators=50,max_depth=20,n_jobs =-1,max_features=5)
RFC.fit(X_train,y_train)
y_pred = RFC.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
df_subm.loc[df_test.index, target] = RFC.predict(df_test)
df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()

In [ ]:
df_subm.to_csv('submission.csv')


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
# bag = BaggingClassifier( base_estimator=DecisionTreeClassifier(), n_estimators=10, n_jobs=-1 )
bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), n_estimators=50,
     max_samples=0.4, max_features=1.0,
      bootstrap=True, bootstrap_features=True, oob_score=True,
        n_jobs=-1, 
         random_state=42,
         verbose=0)
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
accuracy_score(y_test,y_pred)

In [ ]:
#----------------------DecisionTreeClassifier-----------------------------
DTC = DecisionTreeClassifier(max_depth=25,criterion='entropy',max_features=8)
DTC.fit(X_train,y_train)
y_pred = DTC.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
#----CROSS VAL
from sklearn.model_selection import cross_val_score
scores = cross_val_score(DTC, X, y, cv=50)
scores.mean()

In [ ]:
mask = df_skill['skill'].str.contains('identity')

df_skill.loc[mask, 'skill'] = 'identit' 

mask = df_skill['skill'].str.contains('entity')

df_skill.loc[mask, 'skill'] = 'entity framework' 

mask = df_skill['skill'].str.contains('andriod')

df_skill.loc[mask, 'skill'] = 'andriod' 


mask = df_skill['skill'].str.contains('yazılım mühendis', 'software engin')

df_skill.loc[mask, 'skill'] = 'yazılım mühendisliği' 

mask = df_skill['skill'].str.contains( 'software engin')

df_skill.loc[mask, 'skill'] = 'yazılım mühendisliği' 


mask = df_skill['skill'].str.contains('spring')

df_skill.loc[mask, 'skill'] = 'spring framework' 


mask = df_skill['skill'].str.contains('hibernate')

df_skill.loc[mask, 'skill'] = 'hibernate' 

mask = df_skill['skill'].str.contains('web geliştirme')

df_skill.loc[mask, 'skill'] = 'web development' 

mask = df_skill['skill'].str.contains('web development')

df_skill.loc[mask, 'skill'] = 'web development' 



mask = df_skill['skill'].str.contains('ajax')

df_skill.loc[mask, 'skill'] = 'ajax' 

mask = df_skill['skill'].str.contains('node')

df_skill.loc[mask, 'skill'] = 'nodejs' 

mask = df_skill['skill'].str.contains('mongodb')

df_skill.loc[mask, 'skill'] = 'mongodb' 


mask = df_skill['skill'].str.contains('photoshop')

df_skill.loc[mask, 'skill'] = 'photoshop' 


mask = df_skill['skill'].str.contains('ekip')

df_skill.loc[mask, 'skill'] = 'ekip çalışması' 


mask = df_skill['skill'].str.contains('opengl')

df_skill.loc[mask, 'skill'] = 'opengl' 


mask = df_skill['skill'].str.contains('oop')

df_skill.loc[mask, 'skill'] = 'oop' 


mask = df_skill['skill'].str.contains('visual studio')

df_skill.loc[mask, 'skill'] = 'visual studio' 

mask = df_skill['skill'].str.contains('xml')

df_skill.loc[mask, 'skill'] = 'xml' 

mask = df_skill['skill'].str.contains('json')

df_skill.loc[mask, 'skill'] = 'json' 

mask = df_skill['skill'].str.contains('jira')

df_skill.loc[mask, 'skill'] = 'matlab' 
mask = df_skill['skill'].str.contains('kotlin')

df_skill.loc[mask, 'skill'] = 'kotlin' 
mask = df_skill['skill'].str.contains('matlab')

df_skill.loc[mask, 'skill'] = 'matlab' 
mask = df_skill['skill'].str.contains('php')

df_skill.loc[mask, 'skill'] = 'php' 
mask = df_skill['skill'].str.contains('.net')

df_skill.loc[mask, 'skill'] = '.net' 
mask = df_skill['skill'].str.contains('software development', 'yazılım geliştirme')

df_skill.loc[mask, 'skill'] = 'software development'
mask = df_skill['skill'].str.contains('yazılım geliştirme')

df_skill.loc[mask, 'skill'] = 'software development'
mask = df_skill['skill'].str.contains('linux')

df_skill.loc[mask, 'skill'] = 'linux'
mask = df_skill['skill'].str.contains('c+')

df_skill.loc[mask, 'skill'] = 'c+'
mask = df_skill['skill'].str.contains('c#')

df_skill.loc[mask, 'skill'] = 'c#'

mask = df_skill['skill'].str.contains('veritabanı')

df_skill.loc[mask, 'skill'] = 'veritabanı'

mask = df_skill['skill'].str.contains('html')

df_skill.loc[mask, 'skill'] = 'html'

mask = df_skill['skill'].str.contains('pyt')

df_skill.loc[mask, 'skill'] = 'python'


mask = df_skill['skill'].str.contains('office')

df_skill.loc[mask, 'skill'] = 'office'


mask = df_skill['skill'].str.contains('sql')

df_skill.loc[mask, 'skill'] = 'sql'

mask = df_skill['skill'].str.contains('java')

df_skill.loc[mask, 'skill'] = 'java'


mask = df_skill['skill'].str.contains('jq')

df_skill.loc[mask, 'skill'] = 'jquery' 


mask = df_skill['skill'].str.contains('engli')

df_skill.loc[mask, 'skill'] = 'english' 

mask = df_skill['skill'].str.contains('agile')

df_skill.loc[mask, 'skill'] = 'agile' 

mask = df_skill['skill'].str.contains('mühendislik')

df_skill.loc[mask, 'skill'] = 'mühendislik' 

mask = df_skill['skill'].str.contains('bootstrap')

df_skill.loc[mask, 'skill'] = 'bootstrap' 

mask = df_skill['skill'].str.contains('turki')

df_skill.loc[mask, 'skill'] = 'turkish' 

mask = df_skill['skill'].str.contains('design patterns')

df_skill.loc[mask, 'skill'] = 'design patterns' 

mask = df_skill['skill'].str.contains('araştırma')

df_skill.loc[mask, 'skill'] = 'araştırma' 

mask = df_skill['skill'].str.contains('maven')

df_skill.loc[mask, 'skill'] = 'maven' 

mask = df_skill['skill'].str.contains('soap')

df_skill.loc[mask, 'skill'] = 'soap' 

mask = df_skill['skill'].str.contains('software design')

df_skill.loc[mask, 'skill'] = 'software design' 

mask = df_skill['skill'].str.contains('angular')

df_skill.loc[mask, 'skill'] = 'angular' 

mask = df_skill['skill'].str.contains('linq')

df_skill.loc[mask, 'skill'] = 'linq' 

mask = df_skill['skill'].str.contains('tfs')

df_skill.loc[mask, 'skill'] = 'tfs' 

mask = df_skill['skill'].str.contains('liderlik')

df_skill.loc[mask, 'skill'] = 'liderlik' 

mask = df_skill['skill'].str.contains('jenkins')

df_skill.loc[mask, 'skill'] = 'jenkins' 

mask = df_skill['skill'].str.contains('wordpress')

df_skill.loc[mask, 'skill'] = 'wordpress' 

mask = df_skill['skill'].str.contains('veri analizi')

df_skill.loc[mask, 'skill'] = 'veri analizi' 

mask = df_skill['skill'].str.contains('redis')

df_skill.loc[mask, 'skill'] = 'redis' 

mask = df_skill['skill'].str.contains('algorit')

df_skill.loc[mask, 'skill'] = 'algoritma' 

mask = df_skill['skill'].str.contains('unity')

df_skill.loc[mask, 'skill'] = 'unity' 

mask = df_skill['skill'].str.contains('teamwork')

df_skill.loc[mask, 'skill'] = 'teamwork' 

mask = df_skill['skill'].str.contains('takım çalışması')

df_skill.loc[mask, 'skill'] = 'teamwork' 


In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
from imblearn.over_sampling import SMOTE,ADASYN, SVMSMOTE
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import train_test_split

data2=df_train[ftrain]
X=data2.drop(columns=['moved_after_2019']).values
y=data2['moved_after_2019'].values

svm_smote = SMOTE( random_state=101) #,sampling_strategy='minority', k_neighbors=5)
X_svm_smote, y_svm_smote = svm_smote.fit_resample(X,y)

X_train_svm, X_test_svm, y_train, y_test = train_test_split(X_svm_smote,y_svm_smote, test_size=0.30, random_state=101)

sc = StandardScaler()
X_train = sc.fit_transform(X_train_svm)
X_test = sc.transform(X_test_svm)

In [ ]:

import numpy as np
from sklearn.metrics import auc, accuracy_score, confusion_matrix, f1_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
knn_param_grid = {"n_neighbors": np.linspace(1,19,10, dtype = int).tolist(),
                 "weights": ["uniform","distance"],
                 "metric":["euclidean","manhattan"]}

knn = KNeighborsClassifier()
clf = GridSearchCV(knn, param_grid=knn_param_grid, cv = StratifiedKFold(n_splits = 10), 
                   scoring = "accuracy", n_jobs = -1,verbose = 1)
clf.fit(X_train,y_train)
y_pred_knn = clf.predict(X_test)
cm_random_knn = confusion_matrix(y_test,y_pred_knn)
acc_random_knn = accuracy_score(y_test, y_pred_knn)
fscore = f1_score(y_test,y_pred_knn)
result.append(acc_random_knn)

In [ ]:

from lightgbm import LGBMClassifier
import lightgbm as lgb
clf_lgb = lgb.LGBMClassifier()
clf_lgb.fit(X_train, y_train)
y_pred=clf_lgb.predict(X_test)


In [ ]:
print(f'Scores mean: {np.mean(scores):.4f}')
print(f'Scores std: {np.std(scores):.4f}')

In [ ]:
clf.fit(X, y)
df_subm.loc[df_test.index, target] = clf.predict(df_test)

df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()
df_subm.to_csv('submission.csv')


In [ ]:
df_subm.to_csv('submission.csv')


In [ ]:


# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a KNN classifier with k=5 and fit the model to the training data
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Make predictions on the test data
y_pred = knn.predict(X_test)

# Evaluate the accuracy of the model on the test data
score = knn.score(X_test, y_test)
print('KNN model score is %0.4f' % score)


In [ ]:
df_subm.loc[df_test.index, target] = knn.predict(df_test)

df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()


In [ ]:
df_subm.to_csv('submission.csv')


In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)




In [ ]:
scores = cross_val_score(dtc, X, y, cv=cv, scoring='accuracy')
print(f'Scores mean: {np.mean(scores):.4f}')
print(f'Scores std: {np.std(scores):.4f}')

In [ ]:
df_subm.loc[df_test.index, target] = dtc.predict(df_test)

df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()


In [ ]:
df_subm.to_csv('submission.csv')
